<a href="https://colab.research.google.com/github/GGpark1/Deep_Learning_Study/blob/master/seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import shutil
import zipfile

import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [7]:
#파일 import

http = urllib3.PoolManager()
url ='http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:       
    shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

In [10]:
#df 구성

lines = pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del lines['lic']
print('전체 샘플의 개수 :',len(lines))

전체 샘플의 개수 : 192341


In [11]:
lines.head()

,src,tar
0,Go.,Va !
1,Go.,Marche.
2,Go.,Bouge !
3,Hi.,Salut !
4,Hi.,Salut.


In [12]:
lines = lines[0:60000]

In [13]:
lines.sample(10)

,src,tar
2577,We're done.,Nous avons fini.
5284,I'm too busy.,Je suis trop occupée.
22153,Are they worth it?,Valent-ils le coup ?
47437,Everything was stolen.,Tout a été volé.
17078,Are you all nuts?,Êtes-vous toutes dingues ?
52333,We work to earn money.,Nous travaillons pour gagner de l'argent.
11339,Please do this.,"Fais ça, s'il te plaît !"
57803,Sorry to interrupt you.,Désolé de t'interrompre.
28400,Go play in traffic.,Va te faire pendre.
1966,I like art.,J'apprécie l'art.


In [14]:
lines.tar = lines.tar.apply(lambda x : '\t ' + x + ' \n')
lines.sample(10)

,src,tar
38664,We may be in danger.,\t Il se peut que nous soyons en danger. \n
20479,This is required.,\t Il le faut. \n
55276,I don't like your name.,\t Je n'aime pas ton prénom. \n
7103,I dislike you.,\t Je ne t'aime pas. \n
6069,Tom was rich.,\t Tom était riche. \n
22921,He likes sleeping.,\t Il aime dormir. \n
36629,It's slightly windy.,\t Il y a un peu de vent. \n
16060,Tom never jokes.,\t Tom ne fait jamais de blagues. \n
46590,All of them were gone.,\t Ils étaient tous partis. \n
59822,Who are you voting for?,\t Pour qui votes-tu ? \n
